<a href="https://colab.research.google.com/github/abmishra1234/LLM-Apps/blob/Development/GenerateSummaryForYoutubeVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! pip install pytube openai-whisper
#! pip install OpenAI

In [23]:
from pytube import YouTube
import whisper
import os
from openai import OpenAI

In [12]:
import os
from getpass import getpass
OA_token = getpass()
os.environ['OPENAI_API_KEY'] = OA_token

··········


In [5]:
# Function to download YouTube video audio
def download_youtube_audio(youtube_url, output_path):
    yt = YouTube(youtube_url)
    audio_stream = yt.streams.get_audio_only()
    audio_stream.download(output_path=output_path, filename="audio.mp4")
    return os.path.join(output_path, "audio.mp4")  # pytube saves as mp4

In [6]:
# Function to transcribe audio using Whisper
def transcribe_audio(audio_file_path):
    # Load the Whisper model
    model = whisper.load_model("base")

    # Transcribe the audio
    result = model.transcribe(audio_file_path)
    return result['text']

In [18]:
# Function to generate a summary from the transcript
def generate_summary(transcript):
    # Sending the transcript to OpenAI's API
    client = OpenAI()

    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
            {
                "role": "system",
                "content": "Summarize the following transcript and highlight the actionable items and important points:"
            },
            {
                "role": "user",
                "content": transcript
            }
      ]
    )

    # Access the last message in the conversation for the summary
    summary = response.choices[0].message.content
    print(summary)
    return summary.strip()
    # print(completion.choices[0].message.content)

In [7]:
# function to perform the transcribe tasks
def transcribe_youtube_video(youtube_url, docs_path):
    # Ensure the docs_path exists
    if not os.path.exists(docs_path):
        os.makedirs(docs_path)

    # Download the audio from YouTube
    audio_file_path = download_youtube_audio(youtube_url, docs_path)

    # Transcribe the audio to text
    transcript = transcribe_audio(audio_file_path)

    # Save the transcript to a file
    transcript_file_path = os.path.join(docs_path, "transcript.txt")
    with open(transcript_file_path, 'w') as file:
        file.write(transcript)

    # Clean up the audio file if you no longer need it
    os.remove(audio_file_path)

    print("Transcription completed successfully!")
    return transcript_file_path

In [19]:
# Main function into the main workflow
def transcribe_and_summarize_youtube_video(youtube_url, docs_path):
    # Transcribe the YouTube video
    transcript_file_path = transcribe_youtube_video(youtube_url, docs_path)

    # Read the transcript from the file
    with open(transcript_file_path, 'r') as file:
        transcript = file.read()

    # Generate a summary of the transcript
    summary = generate_summary(transcript)

    # Save the summary to a file
    summary_file_path = os.path.join(docs_path, "summary.txt")
    with open(summary_file_path, 'w') as file:
        file.write(summary)

    print("Summary generation completed successfully!")
    return summary_file_path


In [9]:
# ! pip install ffmpeg

In [10]:
#!pip install openai --upgrade

In [ ]:
# Replace 'your_youtube_video_url' with the actual YouTube video URL
youtube_video_url = 'https://www.youtube.com/watch?v=5zPrVf-BlKE&t=71s'
docs_directory = "docs/youtube/"
summary_file = transcribe_and_summarize_youtube_video(youtube_video_url, docs_directory)
print(f"The summary has been saved to: {summary_file}")

